In [18]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from torch.utils.data import Dataset, DataLoader
import os
import matplotlib.pyplot as plt
import csv

import torch
import captum
from captum.attr import IntegratedGradients
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

device = "cpu"

Using device: cuda


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold
import numpy as np

# === JVGAN COMPONENTS ===
class JVGANEncoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(JVGANEncoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, 128, batch_first=True)
        self.fc_mu = nn.Linear(128, latent_dim)
        self.fc_logvar = nn.Linear(128, latent_dim)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        h = h_n.squeeze(0)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        std = torch.exp(0.5 * logvar)
        z = mu + std * torch.randn_like(std)
        return z, mu, logvar

class JVGANDecoder(nn.Module):
    def __init__(self, latent_dim, input_dim, sequence_length=65):
        super(JVGANDecoder, self).__init__()
        self.latent_to_hidden = nn.Linear(latent_dim, 128)
        self.lstm = nn.LSTM(128, 128, batch_first=True)
        self.output_layer = nn.Linear(128, input_dim)
        self.sequence_length = sequence_length

    def forward(self, z):
        h0 = self.latent_to_hidden(z).unsqueeze(1).repeat(1, self.sequence_length, 1)
        lstm_out, _ = self.lstm(h0)
        return self.output_layer(lstm_out)

class JVGANDiscriminator(nn.Module):
    def __init__(self, input_dim):
        super(JVGANDiscriminator, self).__init__()
        self.lstm = nn.LSTM(input_dim, 128, batch_first=True)
        self.classifier = nn.Sequential(
            nn.Linear(128, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        return self.classifier(h_n.squeeze(0))

class JVGAN(nn.Module):
    def __init__(self, input_dim, latent_dim=32, sequence_length=65):
        super(JVGAN, self).__init__()
        self.encoder = JVGANEncoder(input_dim, latent_dim)
        self.decoder = JVGANDecoder(latent_dim, input_dim, sequence_length)
        self.discriminator = JVGANDiscriminator(input_dim)

    def forward(self, x):
        z, mu, logvar = self.encoder(x)
        x_recon = self.decoder(z)
        d_real = self.discriminator(x)
        d_fake = self.discriminator(x_recon.detach())
        return x_recon, d_real, d_fake, mu, logvar


def detect_anomaly(x, model, threshold):
    model.eval()
    with torch.no_grad():
        x_recon, _, _, _, _ = model(x)
        last_real = x[:, -1, :]
        last_recon = x_recon[:, -1, :]
        error = F.mse_loss(last_recon, last_real, reduction='none').mean(dim=1)
        return error > threshold, error

import os
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold

def train_jvgan(dataset, input_dim, num_epochs=30, batch_size=64, k_folds=5, latent_dim=32,
                threshold=0.05, device="cuda" if torch.cuda.is_available() else "cpu", save_dir="jvgan_models"):
    
    os.makedirs(save_dir, exist_ok=True)
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
        print(f"\n=== Fold {fold + 1}/{k_folds} ===")
        
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        model = JVGAN(input_dim=input_dim, latent_dim=latent_dim).to(device)

        # Define separate optimizers
        optimizer_encoder = torch.optim.Adam(model.encoder.parameters(), lr=1e-3)
        optimizer_decoder = torch.optim.Adam(model.decoder.parameters(), lr=1e-3)
        optimizer_discriminator = torch.optim.Adam(model.discriminator.parameters(), lr=1e-4)

        # Learning rate schedulers
        scheduler_encoder = torch.optim.lr_scheduler.StepLR(optimizer_encoder, step_size=10, gamma=0.5)
        scheduler_decoder = torch.optim.lr_scheduler.StepLR(optimizer_decoder, step_size=10, gamma=0.5)
        scheduler_discriminator = torch.optim.lr_scheduler.StepLR(optimizer_discriminator, step_size=10, gamma=0.5)

        for epoch in range(num_epochs):
            model.train()
            total_loss = 0

            for batch_x, in train_loader:
                batch_x = batch_x.to(device)

                # Forward pass
                x_recon, d_real, d_fake, mu, logvar = model(batch_x)
                last_real = batch_x[:, -1, :]
                last_recon = x_recon[:, -1, :]

                # Losses
                recon_loss = F.mse_loss(last_recon, last_real)
                kld = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

                real_labels = torch.ones_like(d_real)
                fake_labels = torch.zeros_like(d_fake)
                d_loss = F.binary_cross_entropy(d_real, real_labels) + F.binary_cross_entropy(d_fake, fake_labels)

                # Total loss
                total = recon_loss + kld + d_loss

                # Backpropagation with multiple optimizers
                optimizer_encoder.zero_grad()
                optimizer_decoder.zero_grad()
                optimizer_discriminator.zero_grad()
                total.backward()
                optimizer_encoder.step()
                optimizer_decoder.step()
                optimizer_discriminator.step()

                total_loss += total.item()

            # Scheduler steps
            scheduler_encoder.step()
            scheduler_decoder.step()
            scheduler_discriminator.step()

            print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {total_loss:.4f}")

        # Save model
        model_path = os.path.join(save_dir, f"jvgan_fold{fold + 1}.pt")
        torch.save(model, model_path)
        print(f"Saved model for fold {fold + 1} at {model_path}")

        # Validation
        model.eval()
        val_data = torch.stack([x[0] for x in val_subset]).to(device)
        print("size of val_data:", val_data.shape)
        print("size of val_subset:", len(val_subset))
        is_anomaly, recon_errors = detect_anomaly(val_data, model, threshold)
        fold_results.append((is_anomaly.cpu(), recon_errors.cpu()))

    return fold_results



def create_sequences(df, look_back, look_forward, CU, DU):
    """
    Create LSTM-ready sequences from a DataFrame.

    Parameters:
        df (pd.DataFrame): Input dataframe.
        look_back (int): Number of timesteps to look back.
        look_forward (int): Number of timesteps to predict forward.

    Returns:
        X (np.ndarray): Shape (samples, look_back, features)
        y_df (pd.DataFrame): Shape (samples, features), from last step of forecast horizon
        column_names (list): List of column names used
    """
    # print(df.columns)
    data = df.drop(columns=[f"srscu{CU}_stepStress", f"srscu{CU}_stressType", f"srsdu{DU}_stepStress", f"srsdu{DU}_stressType"]).values
    targets = df[[f"srscu{CU}_stepStress", f"srscu{CU}_stressType", f"srsdu{DU}_stepStress", f"srsdu{DU}_stressType"]].values

    pca_features_input, pca_features_output, input_targets, output_targets  = [], [], [], []

    for i in range(len(data) - look_back - look_forward):
        pca_features_input.append(data[i:(i + look_back + look_forward)])
        pca_features_output.append(data[i + look_back + look_forward])  # last step of forecast

        input_targets.append(targets[i:(i + look_back + look_forward)])
        output_targets.append(targets[i + look_back + look_forward])  # last step of forecast

    return pca_features_input, pca_features_output, input_targets, output_targets

In [20]:
# import argparse


# parser = argparse.ArgumentParser(description="Run script with optional CU and DU values.")
# parser.add_argument("--cu", type=int, default=0, help="CU value (default: 0)")
# parser.add_argument("--du", type=int, default=0, help="DU value (default: 0)")
# parser.add_argument("--lb", type=int, default=60, help="lookback value (default: 60)")
# parser.add_argument("--lf", type=int, default=1, help="lookforward value (default: 5)")

# args = parser.parse_args()

# CU, DU = args.cu, args.du
# LOOK_BACK, LOOK_FORWARD = args.lb, args.lf

CU = 0
DU = 0

LOOK_BACK, LOOK_FORWARD = 65, 0

In [21]:
# Load dataset
dataset = pd.read_csv(f'dataset_srscu{CU}_srsdu{DU}.csv')
# dataset = dataset[:int(0.01*len(dataset))]

dataset.index = dataset['Timestamp']
dataset = dataset.drop(columns=['Timestamp'])

# dataset.head()


# Imputing

In [22]:
features = dataset.drop(columns=[f"srscu{CU}_stepStress", f"srscu{CU}_stressType", f"srsdu{DU}_stepStress", f"srsdu{DU}_stressType"])
targets = dataset[[f"srscu{CU}_stepStress", f"srscu{CU}_stressType", f"srsdu{DU}_stepStress", f"srsdu{DU}_stressType"]]

# **Data Preprocessing**
# Handle missing values
features = features.apply(lambda x: x.fillna(0) if x.isna().all() else x)


threshold = 0.6 * len(features)
features = features.loc[:, ~features.columns.duplicated()]  # Remove duplicates

for col in features.columns:
    nan_count = features[col].isna().sum()
    if int(nan_count) > threshold:  # Explicit scalar conversion
        mode_value = features[col].mode().iloc[0] if not features[col].mode().empty else 0
        features[col].fillna(mode_value, inplace=True)

numeric_cols = features.select_dtypes(include=[np.number]).columns
features[numeric_cols] = features[numeric_cols].fillna(features[numeric_cols].mean())

# Scale features
import joblib
from pathlib import Path


scaler = joblib.load("minmax_scaler.pkl")
features_scaled = scaler.fit_transform(features)

# Drop the unwanted columns from the original dataset
drop_cols = [f"srscu{CU}_stepStress", f"srscu{CU}_stressType", f"srsdu{DU}_stepStress", f"srsdu{DU}_stressType"]
columns_to_keep = [col for col in dataset.columns if col not in drop_cols]

# Create the DataFrame with scaled features
features_scaled = pd.DataFrame(features_scaled, columns=columns_to_keep, index=dataset.index)

# Concatenate with targets
new_dataset = pd.concat([features_scaled, targets], axis=1)

In [23]:
features_input , features_output, input_targets, output_targets = create_sequences(new_dataset, LOOK_BACK, LOOK_FORWARD, CU, DU)


print(np.array(features_input).shape, np.array(features_output).shape)

print(np.array(input_targets).shape, np.array(output_targets).shape)



(1434, 65, 290) (1434, 290)
(1434, 65, 4) (1434, 4)


In [24]:
tensor_data = torch.tensor(features_input, dtype=torch.float32)

print(tensor_data.shape)

k_folds = 2
results = train_jvgan(TensorDataset(tensor_data), input_dim=np.array(features_input).shape[2], num_epochs=2, batch_size=64, k_folds=k_folds, device=device)

torch.Size([1434, 65, 290])

=== Fold 1/2 ===
Epoch 1/2 - Loss: 25.3031
Epoch 2/2 - Loss: 18.9685
Saved model for fold 1 at jvgan_models/jvgan_fold1.pt
size of val_data: torch.Size([717, 65, 290])
size of val_subset: 717

=== Fold 2/2 ===
Epoch 1/2 - Loss: 25.2167
Epoch 2/2 - Loss: 18.6295
Saved model for fold 2 at jvgan_models/jvgan_fold2.pt
size of val_data: torch.Size([717, 65, 290])
size of val_subset: 717


In [25]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score
import pickle  # or use joblib if preferred


# features_input = pd.DataFrame(features_input, columns=dataset.drop(columns=[f"srscu{CU}_stepStress", f"srscu{CU}_stressType", f"srsdu{DU}_stepStress", f"srsdu{DU}_stressType"]).columns, index=dataset.index)
# input_targets = pd.DataFrame(input_targets, columns=[f"srscu{CU}_stepStress", f"srscu{CU}_stressType", f"srsdu{DU}_stepStress", f"srsdu{DU}_stressType"], index = dataset.index)

def find_optimal_threshold(model, dataset, true_labels, thresholds=None, device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Args:
        model: Trained JVGAN model.
        dataset: Tensor of shape (N, 65, features).
        true_labels: Ground truth binary anomaly labels for each sequence (0: normal, 1: anomaly).
        thresholds: List or numpy array of threshold values to try.
        device: Device to run inference on.
        
    Returns:
        best_threshold: Threshold giving best F1-score.
        metrics_dict: Dictionary of threshold to (F1, precision, recall).
    """
    if thresholds is None:
        thresholds = np.linspace(0.001, 0.5, 100)

    model.eval()
    dataset = torch.tensor(dataset, dtype=torch.float32).to(device)

    # Get reconstruction errors
    with torch.no_grad():
        x_recon, _, _, _, _ = model(dataset)
        last_real = dataset[:, -1, :]
        last_recon = x_recon[:, -1, :]
        errors = F.mse_loss(last_recon, last_real, reduction='none').mean(dim=1).cpu().numpy()

    true_labels = np.array(true_labels)
    best_f1 = -1
    best_threshold = None
    metrics_dict = {}

    for thresh in thresholds:
        preds = (errors > thresh).astype(int)
        f1 = f1_score(true_labels, preds)
        precision = precision_score(true_labels, preds, zero_division=0)
        recall = recall_score(true_labels, preds, zero_division=0)

        metrics_dict[thresh] = (f1, precision, recall)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = thresh

    print(f"Best Threshold: {best_threshold:.4f} | F1: {best_f1:.4f} | Precision: {metrics_dict[best_threshold][1]:.4f} | Recall: {metrics_dict[best_threshold][2]:.4f}")
    return best_threshold, metrics_dict


save_metrics_dir = "jvgan_thresholds"
os.makedirs(save_metrics_dir, exist_ok=True)

for i in range(1, k_folds + 1):
    # Load model using torch.load
    model_path = f"jvgan_models/jvgan_fold{i}.pt"
    JVGAN_model = torch.load(model_path, map_location=device)
    
    # Get true labels from input_targets
    true_labels = []
    for sample in input_targets:  # i-1 because Python is 0-indexed
        last_sample_target = sample[-1][:]
        last_sample_target = last_sample_target.reshape(1, 4)
        last_sample_target = pd.DataFrame(last_sample_target, columns=[f"srscu{CU}_stepStress", f"srscu{CU}_stressType", f"srsdu{DU}_stepStress", f"srsdu{DU}_stressType"])
        if (last_sample_target[f'srscu{CU}_stressType'].item() != 0 or last_sample_target[f'srsdu{DU}_stressType'].item() != 0):
            true_labels.append(1)
        else:
            true_labels.append(0)
    
    # Compute best threshold
    best_threshold, metrics_dict = find_optimal_threshold(
        JVGAN_model,
        dataset=features_input,  # Assuming features_input is a list of tensors per fold
        true_labels=true_labels,
        thresholds=np.linspace(0.001, 0.5, 100),
        device=device
    )

    # Save best threshold and metrics_dict to a file
    results = {
        "fold": i,
        "best_threshold": best_threshold,
        "metrics_dict": metrics_dict
    }

    with open(Path(save_metrics_dir) / f"threshold_metrics_fold{i}.pkl", "wb") as f:
        pickle.dump(results, f)
    with open(Path(save_metrics_dir) / f"threshold_metrics_fold{i}.txt", "w") as f:
        f.write(str(results))

    print(f"Saved threshold metrics for Fold {i} at {save_metrics_dir}/threshold_metrics_fold{i}.pkl")


/tmp/ipykernel_14644/4030787430.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  JVGAN_model = torch.load(model_path, map_location=device)


Best Threshold: 0.0010 | F1: 0.9037 | Precision: 0.8243 | Recall: 1.0000
Saved threshold metrics for Fold 1 at jvgan_thresholds/threshold_metrics_fold1.pkl


/tmp/ipykernel_14644/4030787430.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  JVGAN_model = torch.load(model_path, map_location=device)


Best Threshold: 0.0010 | F1: 0.9037 | Precision: 0.8243 | Recall: 1.0000
Saved threshold metrics for Fold 2 at jvgan_thresholds/threshold_metrics_fold2.pkl
